In [2]:
import re
import json
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

In [8]:
session = requests.Session()
session.headers.update({
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) \
                   AppleWebKit/605.1.15 (KHTML, like Gecko) \
                   Chrome/100.0.4896.127 Safari/605.1.15 Firefox/100.0" 
    })

In [ ]:
results = {}

# Quote

In [ ]:
def get_short_name(ticker):
    """
    This function returns the company's name
    """
    url = f"http://performance.morningstar.com/stock/performance-return.action?t={ticker}"

    response = session.get(url).text
    soup = BeautifulSoup(response)

    short_name = soup.find("h1").get_text()

    return short_name

In [ ]:
def get_quote_data(ticker):
    """
    This function returns the following quote data:
    Last Price, Daily Change, Daily Change (%)
    """
    url = f"http://performance.morningstar.com/perform/Performance/stock/quote-data-strip.action?&t={ticker}"

    response = session.get(url).text
    soup = BeautifulSoup(response)

    data = soup.find_all("span", attrs={"class": "data_big"})
    last_price = float(re.search("[0-9\.\-]+", data[0].get_text()).group())
    daily_change = float(re.search("[0-9\.\-]+", data[1].get_text()).group())
    daily_pct_change = float(re.search("[0-9\.\-]+", data[2].get_text()).group())

    quote_data = {
                  "Last Price": last_price,
                  "Daily Change": daily_change,
                  "Daily Change (%)": daily_pct_change
                 }

    return quote_data

# Performance

## Total Returns

In [ ]:
def get_performance(ticker, data="ph"):
    """
    This function returns the following performance tables:
    1) ph = Performance History
    2) ttr = Trailing Total Returns
    """
    if data == "ph":
        url = f"http://performance.morningstar.com/perform/Performance/stock/performance-history.action?&t={ticker}&s=0P00001MK8&y=10"
    elif data == "ttr":
        url = f"http://performance.morningstar.com/perform/Performance/stock/trailing-total-returns.action?&t={ticker}&s=0P00001MK8&align=d"
    else:
        return None

    df = pd.read_html(url, header=0, index_col=0)[0]
    df = df.apply(lambda x: pd.to_numeric(x, errors="coerce"))

    return df

## Price History

In [3]:
def get_price_history(ticker):
    """
    This function returns the historical prices of the company
    """
    url = f"http://performance.morningstar.com/perform/Performance/stock/price-history.action?&t={ticker}&pd=max"

    df = pd.read_html(url, header=0, index_col=0, keep_default_na=False, parse_dates=True)[0]
    df["Volume"] = df["Volume"].apply(lambda x: x.replace("Mil", ""))
    df = df.apply(lambda x: pd.to_numeric(x, errors="coerce"))
    df.index.name = ""

    df.dropna(axis=0, how="all", inplace=True)

    return df

## Dividends

In [ ]:
def get_annual_dividends(ticker):
    """
    This function returns the company's annual dividends
    """
    url = f"http://performance.morningstar.com/perform/Performance/stock/annual-dividends.action?&t={ticker}"

    df = pd.read_html(url, header=0, index_col=0)[0]
    df = df.apply(lambda x: pd.to_numeric(x, errors="coerce"))
    df = df.loc[["Dividend Amount", "Year-end Yield %"]]

    return df

In [ ]:
def get_dividend_history(ticker):
    """
    This function returns the company's dividend history
    """
    url = f"http://performance.morningstar.com/perform/Performance/stock/dividend-history.action?&t={ticker}"

    response = session.get(url).text
    soup = BeautifulSoup(response)

    th_str = soup.find_all("th", attrs={"class": "str"})
    cols = [col.get_text() for col in th_str]

    td = soup.find_all("td")
    rows = [row.get_text() for row in td]
    rows = [rows[i: i+len(cols)] for i in range(0, len(rows), len(cols))]

    df = pd.DataFrame(data=rows, columns=cols)
    cols_to_dt = ["Ex-Dividend Date", "Declaration Date", "Record Date", "Payable Date"]
    df[cols_to_dt] = df[cols_to_dt].apply(pd.to_datetime)
    df["Amount"] = df["Amount"].apply(lambda x: float(x.replace("$", "")))

    return df

# Key Ratios

## Financials

In [ ]:
def get_key_finanacial_ratios(ticker):
    """
    This function returns key financial ratios 
    """
    url = f"http://financials.morningstar.com/finan/financials/getFinancePart.html?&t={ticker}"

    response = session.get(url).json()
    soup = BeautifulSoup(response["componentData"])

    th_col = soup.find_all("th", attrs={"scope": "col"})
    cols = [col.get_text() for col in th_col]

    th_row = soup.find_all("th", attrs={"scope": "row"})
    lbls = [lbl.get_text().replace(" *", "").replace("\xa0", " ") for lbl in th_row][1:]

    td = soup.find_all("td")
    rows = [row.get_text().replace(",", "") for row in td if row.get_text()][:-1]
    rows = [rows[i: i+len(cols)] for i in range(0, len(rows), len(cols))]

    df = pd.DataFrame(data=rows, columns=cols, index=lbls)
    df = df.apply(lambda x: pd.to_numeric(x, errors="coerce"))

    return df

## Key Ratios

In [ ]:
def get_total_key_ratios_data(ticker):
    """
    This function returns the necessary data to build a Key Ratios statement
    """
    url = f"http://financials.morningstar.com/finan/financials/getKeyStatPart.html?&t={ticker}"

    response = session.get(url).json()
    soup = BeautifulSoup(response["componentData"])

    rowspan = 11

    th_col = soup.find_all("th", attrs={"scope": "col"})
    cols = [col.get_text() for col in th_col]

    th_row = soup.find_all("th", attrs={"scope": "row"})
    lbls = [lbl.get_text().replace(" *", "").replace("\xa0", " ") for lbl in th_row]

    td = soup.find_all("td")
    rows = [row.get_text().replace(",", "") for row in td if row.get_text()]
    rows = [rows[i: i+rowspan] for i in range(0, len(rows), rowspan)]

    return cols, lbls, rows

### Margins % of Sales

In [ ]:
def get_margins(ticker):
    """
    This function returns the Margins % of Sales ratios
    """
    cols, lbls, rows = get_total_key_ratios_data(ticker)
 
    df = pd.DataFrame(data=rows[:9], columns=cols[1:12], index=lbls[:9])
    df = df.apply(lambda x: pd.to_numeric(x, errors="coerce"))

    return df

### Profitability

In [ ]:
def get_profitability(ticker):
    """
    This function returns the Profitability ratios
    """
    cols, lbls, rows = get_total_key_ratios_data(ticker)
 
    df = pd.DataFrame(data=rows[9:17], columns=cols[13:24], index=lbls[9:17])
    df = df.apply(lambda x: pd.to_numeric(x, errors="coerce"))

    return df

### Growth

In [ ]:
def get_growth(ticker):
    """
    This function returns the Growth ratios
    """
    cols, lbls, rows = get_total_key_ratios_data(ticker)

    rowspan = 11
    df_rows = rows[17:33]

    for i in [0, 5, 10, 15]:
        df_rows.insert(i, ["—"] * rowspan)

    df = pd.DataFrame(data=df_rows, columns=cols[24:35], index=lbls[17:37])
    df = df.apply(lambda x: pd.to_numeric(x, errors="coerce"))

    return df

### Cash Flow

In [ ]:
def get_cash_flow_ratios(ticker):
    """
    This function returns the Cash Flow Ratios
    """
    cols, lbls, rows = get_total_key_ratios_data(ticker)
 
    df = pd.DataFrame(data=rows[33:38], columns=cols[36:47], index=lbls[37:42])
    df = df.apply(lambda x: pd.to_numeric(x, errors="coerce"))

    return df

### Balance Sheet Items (in %)

In [ ]:
def get_balance_sheet_items(ticker):
    """
    This function returns the Balance Sheet Items (in %)
    """
    cols, lbls, rows = get_total_key_ratios_data(ticker)
 
    df = pd.DataFrame(data=rows[38:58], columns=cols[48:59], index=lbls[42:62])
    df = df.apply(lambda x: pd.to_numeric(x, errors="coerce"))

    return df

### Liquidity/Financial Health

In [ ]:
def get_liquidity(ticker):
    """
    This function returns the Liquidity/Financial Health ratios
    """
    cols, lbls, rows = get_total_key_ratios_data(ticker)
 
    df = pd.DataFrame(data=rows[58:62], columns=cols[60:71], index=lbls[62:66])
    df = df.apply(lambda x: pd.to_numeric(x, errors="coerce"))

    return df

### Efficiency

In [ ]:
def get_efficiency(ticker):
    """
    This function returns the Efficiency ratios
    """
    cols, lbls, rows = get_total_key_ratios_data(ticker)
 
    df = pd.DataFrame(data=rows[62:], columns=cols[72:], index=lbls[66:])
    df = df.apply(lambda x: pd.to_numeric(x, errors="coerce"))

    return df

# Financials

In [ ]:
def get_financials(ticker, statement="is"):
    """
    This function returns the following financial statements:
    1) is = Income Statement
    2) bs = Balance Sheet
    3) cf = Cash Flow
    """
    if any(statement in st for st in ["is", "bs", "cf"]):
        url = f"http://financials.morningstar.com/ajax/ReportProcess4HtmlAjax.html?&t={ticker}&reportType={statement}"
    else:
        return None

    response = session.get(url).json()
    soup = BeautifulSoup(response["result"])

    cols = soup.find_all("div", attrs={"class": "year"})
    cols = [col.get_text() for col in cols]

    lbls = soup.find_all("div", attrs={"class": "lbl"})[1:]
    alts = soup.find_all("div", title=True, attrs={"class": "lbl"})

    lbls = [lbl.get_text() for lbl in lbls]
    lbls = list(filter(lambda x : x != "\xa0", lbls))

    for i, lbl in enumerate(lbls):
        for j, alt in enumerate(alts):
            if lbl == alt.get_text():
                lbls[i] = alts[j]["title"]

    rows = soup.find_all("div", attrs={"class": "pos"})
    rows = [row.get_text().replace(",", "").replace("(", "-").replace(")", "") for row in rows]
    rows = [rows[i: i+len(cols)] for i in range(0, len(rows), len(cols))]

    df = pd.DataFrame(data=rows, columns=cols, index=lbls)
    df = df.apply(lambda x: pd.to_numeric(x, errors="coerce"))

    return df


# Valuation

In [ ]:
def get_valuation(ticker, table="fv"):
    """
    This function returns the following valuation tables:
    1) cv = Current Valuation
    2) fv = Forward Valuation
    3) fc = Forward Comparisons
    """
    if table == "cv":
        url = f"http://financials.morningstar.com/valuate/current-valuation-list.action?&t={ticker}"
    elif table == "fv":
        url = f"http://financials.morningstar.com/valuate/forward-valuation-list.action?&t={ticker}"
    elif table == "fc":
        url = f"http://financials.morningstar.com/valuate/forward-comparisons-list.action?&t={ticker}"
    else:
        return None

    df = pd.read_html(url, header=0, index_col=0)[0]
    df = df.apply(lambda x: pd.to_numeric(x, errors="coerce"))

    df.dropna(axis=0, how="all", inplace=True)
    df.dropna(axis=1, how="all", inplace=True)

    return df

In [ ]:
def get_valuation_history(ticker):
    """
    This function returns the company's valuation history
    """
    url = f"http://financials.morningstar.com/valuate/valuation-history.action?&t={ticker}&type=price-earnings"

    df = pd.read_html(url, index_col=0)[0]
    df = df.apply(lambda x: pd.to_numeric(x, errors="coerce"))
    df.columns = get_key_finanacial_ratios(ticker).columns
    df.index.name = ""

    return df

# 8 Pillars Model

In [ ]:
def get_eight_pillars(ticker):
    """
    This function returns 8 Pillars Model results
    """
    income_statement = get_financials(ticker, "is")

    pe_ratio = get_valuation_history(ticker).loc[:"Price/Book"].loc[ticker][5:10]
    avg_pe_ratio = pe_ratio.mean()

    roic = get_profitability(ticker).loc["Return on Invested Capital %"][5:10]
    avg_roic = roic.mean()

    revenue = income_statement.loc["Revenue"][:-1]
    revenue_change = revenue[-1] - revenue[0]

    net_income = income_statement.loc["Net income"][:-1]
    net_income_change = net_income[-1] - net_income[0]

    shares_outstanding = income_statement.loc["Weighted average shares outstanding":].loc["Diluted"][:-1]
    shares_outstanding_change = (shares_outstanding[-1] / shares_outstanding[0] - 1) * 100

    free_cash_flow = get_financials(ticker, "cf").loc["Free cash flow"][:-1]
    free_cash_flow_change = free_cash_flow[-1] - free_cash_flow[0]
    avg_free_cash_flow = free_cash_flow.mean()

    long_term_liabilities = get_financials(ticker, "bs").loc["Total non-current liabilities"][-1]
    ltl_to_fcf = long_term_liabilities / avg_free_cash_flow

    current_price = get_quote_data(ticker)["Last Price"]
    market_cap = shares_outstanding[-1] * current_price
    price_to_fcf = market_cap / avg_free_cash_flow

    pillar_1 = "✔️" if avg_pe_ratio < 22.5 else "❌"
    pillar_2 = "✔️" if avg_roic > 9 else "❌"
    pillar_3 = "✔️" if revenue_change > 0 else "❌"
    pillar_4 = "✔️" if net_income_change > 0 else "❌"
    pillar_5 = "✔️" if shares_outstanding_change < 0 else "❌"
    pillar_6 = "✔️" if ltl_to_fcf < 5 else "❌"
    pillar_7 = "✔️" if free_cash_flow_change > 0 else "❌"
    pillar_8 = "✔️" if price_to_fcf < 22.5 else "❌"

    print(f"{pillar_1} Pillar #1: 5-Year P/E Ratio < 22.5            = {avg_pe_ratio:.2f}")
    print(f"{pillar_2} Pillar #2: 5-Year ROIC > 9%                   = {avg_roic:.2f}%")
    print(f"{pillar_3} Pillar #3: 5-Year Revenue Growth (Mil)        = {int(revenue_change)}$")
    print(f"{pillar_4} Pillar #4: 5-Year Net Income Growth (Mil)     = {int(net_income_change)}$")
    print(f"{pillar_5} Pillar #5: 5-Year Shares Outstanding (Mil)    = {shares_outstanding_change:.2f}%")
    print(f"{pillar_6} Pillar #6: 5 Year LTL to FCF < 5              = {ltl_to_fcf:.2f}")
    print(f"{pillar_7} Pillar #7: 5-Year Free Cash Flow Growth (Mil) = {int(free_cash_flow_change)}$")
    print(f"{pillar_8} Pillar #8: 5-Year Price to FCF < 22.5         = {price_to_fcf:.2f}")

# Discounted Cash Flow (DCF) Model

In [ ]:
def get_intrinsic_value(ticker):
    """
    This function returns an intrinsic value of the stock
    based on the company's fundamentals.
    P/FCF (Price to Free Cash Flow) is used as a terminal multiplier
    """
    company_name = get_short_name(ticker)
    industry = get_performance(ticker).index[1]
    current_price = get_quote_data(ticker)["Last Price"]

    free_cash_flow = get_financials(ticker, "cf").loc["Free cash flow"][-1]

    if free_cash_flow > 0:
        growth_rates = {}
        forward_free_cash_flow = {}
        npv_free_cash_flow = {}

        estimated_growth_rate = get_valuation(ticker, "fc").loc[ticker][0] / 100
    
        for i in range(1, 11):
            growth_rates[i] = growth_rates[i - 1] * (1.0 - GROWTH_DECLINE_RATE) \
                              if i > 1 else estimated_growth_rate

            forward_free_cash_flow[i] = forward_free_cash_flow[i - 1] * (1.0 + growth_rates[i]) \
                                        if i > 1 else free_cash_flow * (1.0 + growth_rates[i])

            npv_free_cash_flow[i] = forward_free_cash_flow[i] / (1.0 + DISCOUNT_RATE) ** i

        shares_outstanding = get_key_finanacial_ratios(ticker).loc["Shares Mil"][-1]
        market_cap = current_price * shares_outstanding

        balance_sheet = get_financials(ticker, "bs")

        short_term_debt = balance_sheet.loc["Short-term debt"][-1]
        long_term_debt = balance_sheet.loc["Long-term debt"][-1]
        cash_and_equivalents = balance_sheet.loc["Cash and cash equivalents"][-1]
        total_debt = np.nansum([short_term_debt, long_term_debt])
        net_debt = total_debt - cash_and_equivalents

        total_npv_free_cash_flow = sum(npv_free_cash_flow.values())
        terminal_multiple = market_cap / free_cash_flow
        terminal_value = npv_free_cash_flow[10] * terminal_multiple
        equity_value = total_npv_free_cash_flow + terminal_value - net_debt

        intrinsic_value = equity_value / shares_outstanding

        declined_growth_rate = growth_rates[10]

        print(f"Company Ticker                           =  {ticker}")
        print(f"Company Name                             =  {company_name}")
        print(f"Industry                                 =  {industry}")
        print("-" * 60)
        print(f"Free Cash Flow (Mil)                     =  {int(free_cash_flow)}$")
        print(f"Net Debt (Mil)                           =  {int(net_debt)}$")
        print(f"Shares Outstanding (Mil)                 =  {int(shares_outstanding)}")
        print(f"Estimated Growth Rate (+5Y)              =  {round(estimated_growth_rate * 100, 2)}%")
        print(f"Growth Decline Rate                      =  {round(GROWTH_DECLINE_RATE * 100, 2)}%")
        print(f"Declined Estimated Growth Rate (10Y)     =  {round(declined_growth_rate * 100, 2)}%")
        print(f"Discount Rate                            =  {round(DISCOUNT_RATE * 100, 2)}%")
        print(f"Terminal Multiple                        =  {terminal_multiple:.2f}")
        print("-" * 60)
        print(f"Total NPV FCF (Mil)                      =  {total_npv_free_cash_flow:.2f}$")
        print(f"Terminal Value (Mil)                     =  {terminal_value:.2f}$")
        print(f"Enterprise Value (Mil)                   =  {equity_value:.2f}$")
        print("-" * 60)
        print(f"Current Price                            =  {current_price}$")
        print(f"Intrinsic Value                          =  {intrinsic_value:.2f}$")
    else:
        intrinsic_value = np.nan

        print(f"\"{company_name}\" has a negative Free Cash Flow!\n Use other valuation methods.")

    results[ticker] = {
                       "Company Name": company_name,
                       "Industry": industry,
                       "Current Price": current_price,
                       "Intrinsic Value": round(intrinsic_value, 2)
                      }

    return json.dumps(results, indent=4)

# Company Analysis & Evaluation

In [ ]:
"""
This block contains different user inputs
"""

# Ticker Symbol
ticker = "AAPL"

# 5% of growth decline rate will be used to calculate a real free cash flow growth each year for the next 10 years
GROWTH_DECLINE_RATE = 0.05

# Discount Rate
DISCOUNT_RATE = 0.15

# Get Analysis
get_eight_pillars(ticker)
print("-" * 60)
intrinsic_value = get_intrinsic_value(ticker)
results.update(json.loads(intrinsic_value))

❌ Pillar #1: 5-Year P/E Ratio < 22.5            = 25.68
✔️ Pillar #2: 5-Year ROIC > 9%                   = 30.37%
✔️ Pillar #3: 5-Year Revenue Growth (Mil)        = 136583$
✔️ Pillar #4: 5-Year Net Income Growth (Mil)     = 46329$
✔️ Pillar #5: 5-Year Shares Outstanding (Mil)    = -19.72%
✔️ Pillar #6: 5 Year LTL to FCF < 5              = 2.39
✔️ Pillar #7: 5-Year Free Cash Flow Growth (Mil) = 42150$
❌ Pillar #8: 5-Year Price to FCF < 22.5         = 35.06
------------------------------------------------------------
Company Ticker                           =  AAPL
Company Name                             =  Apple Inc
Industry                                 =  Consumer Electronics
------------------------------------------------------------
Free Cash Flow (Mil)                     =  105793$
Net Debt (Mil)                           =  89779$
Shares Outstanding (Mil)                 =  16585
Estimated Growth Rate (+5Y)              =  9.06%
Growth Decline Rate                      =  5.0

In [5]:
tickers = [
           "AAPL", "MSFT", "AMZN", "GOOGL", "FB", "INTC", "AMD", "MU", "IBM", "HPQ",
           "COST", "LOW", "TGT", "WMT", "HD", "BBY", "HON", "KO", "PEP", "MCD",
           "XOM", "CVX", "MA", "V", "PYPL", "CAT", "DE", "MMM", "TJX", "WM",
           "UNH", "ABBV", "ABT", "AMGN", "MRK", "PFE", "LLY", "TMO", "BMY", "PG",
           "JNJ", "CL", "NKE"
           ]

def get_prices(tickers):
    df = pd.DataFrame()
    for ticker in tickers:
        df[ticker] = get_price_history(ticker).loc[:, "Close"]
    return df

get_prices(tickers).to_csv("value-portfolio-stock-prices.csv")

In [ ]:
pd.DataFrame.from_dict(data=results).T #.to_csv("dcf-model-results.csv", index=ticker)